https://data-scientist-brian-kim.tistory.com/84


https://compmath.korea.ac.kr/deeplearning/BackPropagation.html


https://velog.io/@dscwinterstudy%EB%B0%91%EB%B0%94%EB%8B%A5%EB%B6%80%ED%84%B0-%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94-%EB%94%A5%EB%9F%AC%EB%8B%9D-5%EC%9E%A5-n3k5xl3thn


# 곱셈 계층 구현
* `forward`(순전파) : `x * y`
* `backward`(역전파) : `dx = 미분값 * y`, `dy = 미분값 * x`
* 비고 : forward시에 입력되었던 `x, y`를 저장하고 있어야 한다. 그래야 Backward 할 때 반대방향으로 곱해줄 수 있다.

In [1]:
class MulLayer:

  # 레이어에서 사용할 변수 준비
  def __init__(self):
    self.x = None
    self.y = None

  # 곱셈 레이어에서의 순전파 : x, y를 곱하고, 각 x, y를 저장
  def forward(self, x, y):
    self.x = x
    self.y = y

    # 순전파에서 해야 할 연산
    out = x * y
    return out

  # 곱셈 레이어의 역전파 : 들어온 미분 값을 x에는 y를 곱하고, y에는 x를 곱함
  def backward(self, dout):
    # dout : 다음 층에서 흘러들어오는 미분값
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

In [3]:
# 곱셈 레이어 테스트
apple = 100 # 사과 한개 당 가격
applt_cnt = 2 # 사과 개수
tax = 1.1

mul_apple_layer = MulLayer() # 사과의 총 가격을 구하는 레이어
mul_tax_layer = MulLayer() # 소비세까지 적용시킨 가격을 구하는 레이어

# 순전파 수행
#  - 순서를 반드시 잘 지켜야 한다.
#  계획한 순서 그대로 레이어를 배치해서 입력값을 흘려보내야 한다.
#  순전파 수행 시에 A - B - C 순으로 계산을 했다면,
#  역전파 수행 시에는 C - B- A 순으로 미분값을 전달해야 한다.

apple_price = mul_apple_layer.forward(apple, applt_cnt)
price = mul_tax_layer.forward(apple_price, tax)

print("사과의 최종 가격 : {:.0f}".format(price))

사과의 최종 가격 : 220


In [4]:
# 역전파
#  제일 마지막 값에 대한 미분값 생각하기
#  d돈통 / d포스기 = 1
dprice = 1

dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("사과 전체 가격에 대한 미분값 d돈통/d사과전체가격 : {}".format(dapple_price))
print("사과 1개 가격에 대한 미분값 d돈통/d사과1개가격 : {}".format(dapple))
print("사과 개수에 대한 미분값 d돈통/d사과개수 : {}".format(dapple_cnt))
print("소비세에 대한 미분값 d돈통/d소비세 : {}".format(dtax))

사과 전체 가격에 대한 미분값 d돈통/d사과전체가격 : 1.1
사과 1개 가격에 대한 미분값 d돈통/d사과1개가격 : 2.2
사과 개수에 대한 미분값 d돈통/d사과개수 : 110.00000000000001
소비세에 대한 미분값 d돈통/d소비세 : 200


# 덧셈 레이어 구현
* `forward` : `x + y`
* `backward` : 뒷층에서 보낸 미분값에 *1만 하면 된다. `dx = dout * 1`, `dy = dout * 1`
  * `dx = dout * dforward/dx`
  * `dy = dout * dforward/dy`
* 비고 : 곱셈 계층과는 다르게 `forward`시에 입력된 값을 가지고 있지 않아도 된다. 역전파 시에는 미분값만 리턴하면 되니까

In [5]:
class AddLayer:
  def __init__(self):
    pass

  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    dx = dout * 1
    dy = dout * 1

    return dx, dy

In [6]:
apple = 100
apple_cnt = 2

orange = 150
orange_cnt = 3

tax = 1.1

In [8]:
# 1 계층 - 사과에 대한 국소적 계산, 오렌지에 대한 국소적 계산
#  (사과 1개 가격 * 사과 개수), (오렌지 1개 가격 * 오렌지 개수)
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()

# 2계층 - 사과 전체 가격 + 오렌지 전체 가격
add_apple_orange_layer = AddLayer()

# 3계층 - 소비세 적용
mul_tax_layer = MulLayer()

In [10]:
# 순전파

# 1계층 계산
apple_price = mul_apple_layer.forward(apple, apple_cnt)
orange_price = mul_orange_layer.forward(orange, orange_cnt)

# 2계층 계산
total_price = add_apple_orange_layer.forward(apple_price, orange_price)

# 3계층 계산
price = mul_tax_layer.forward(total_price, tax)

In [11]:
price

715.0000000000001

In [13]:
# 역전파
dprice = 1 # d돈통 / d포스기

# 3계층
# d돈통/dtotal_price, d돈통/dtax
dtotal_price, dtax = mul_tax_layer.backward(dprice)

# 2계층
# d돈통 / dapple_price, d돈통 / dorange_price
dapple_price, dorange_price = add_apple_orange_layer.backward(dtotal_price)

# 1계층
#  사과와 오렌지에 대한 각각의 미분값 (국소적 미분)
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)
dorange, dorange_cnt = mul_orange_layer.backward(dorange_price)

print("소비세 미분 : {}".format(dtax))

print("사과 전체 가격 미분 : {}".format(dapple_price))
print("사과 개수 미분 : {}".format(dapple_cnt))
print("사과 가격 미분 : {}".format(dapple))

print("오렌지 전체 가격 미분 : {}".format(dorange_price))
print("오렌지 개수 미분 : {}".format(dorange_cnt))
print("오렌지 가격 미분 : {}".format(dorange))

소비세 미분 : 650
사과 전체 가격 미분 : 1.1
사과 개수 미분 : 110.00000000000001
사과 가격 미분 : 2.2
오렌지 전체 가격 미분 : 1.1
오렌지 개수 미분 : 165.0
오렌지 가격 미분 : 3.3000000000000003
